<a href="https://colab.research.google.com/github/chenmoneygithub/finetune-bert-with-HF-Keras/blob/main/Fine_tune_HuggingFace_model_in_Keras_with_plain_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.9 MB/s 
     |████████████████████████████████| 895 kB 15.5 MB/s 
     |████████████████████████████████| 59 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 17.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Load transformer model and its tokenizer from HuggingFace

In [ ]:
from transformers import DistilBertTokenizerFast, TFDistilBertModel
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
bert = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
text = "This was an absolutely terrible movie!"
encoded_input = tokenizer(text, return_tensors='tf')
output = bert(encoded_input)
print(encoded_input)
print(output)

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[ 101, 2023, 2001, 2019, 7078, 6659, 3185,  999,  102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(1, 9, 768), dtype=float32, numpy=
array([[[ 0.10105765,  0.08145499,  0.14580603, ..., -0.17318317,
          0.41397157,  0.3861206 ],
        [-0.08768288, -0.30625236, -0.00278953, ..., -0.51468325,
          1.0388579 ,  0.3485065 ],
        [ 0.05506645, -0.41134796,  0.00242783, ..., -0.21425858,
          0.50868523,  0.6276908 ],
        ...,
        [ 0.5169393 , -0.19129173, -0.18188265, ..., -0.13861606,
          0.32506967, -0.1977762 ],
        [ 0.2934267 ,  0.08480907,  0.11095915, ...,  0.05719552,
          0.54716593, -0.01751508],
        [ 0.9839468 ,  0.33199084, -0.25651062, ...,  0.10096597,
         -0.20585994, -0.22742088]]], dtype=float32)>, hidden_

Build the text classification model on top of pretrained BERT.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

In [ ]:
class TextClassificationModel(keras.Model):
  def __init__(self, encoder):
    super(TextClassificationModel, self).__init__()
    self.encoder = encoder
    self.encoder.trainable = True
    self.dropout1 = layers.Dropout(0.1)
    self.dropout2 = layers.Dropout(0.1)
    self.dense1 = layers.Dense(20, activation="relu")
    self.dense2 = layers.Dense(2, activation='softmax')
  
  def call(self, input):
    x = self.encoder(input)
    x = x['last_hidden_state'][:, 0, :]
    x = self.dropout1(x)
    x = self.dense1(x)
    x = self.dropout2(x)
    x = self.dense2(x)
    return x

In [ ]:
text_classification_model = TextClassificationModel(bert)

Load IMDB review dataset and convert it into tensorflow dataset.

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2021-11-12 21:53:21--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  24.2MB/s    in 3.3s    

2021-11-12 21:53:24 (24.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length", max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding="max_length", max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length", max_length=512)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
text_classification_model(next(iter(train_dataset.batch(4))))

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.68117476, 0.31882524],
       [0.66069627, 0.3393038 ],
       [0.60794955, 0.3920505 ],
       [0.67073065, 0.32926935]], dtype=float32)>

In [ ]:
text_classification_model.compile(
    tf.keras.optimizers.Adam(learning_rate=5e-5), 
    "sparse_categorical_crossentropy", 
    metrics=["accuracy"])
history = text_classification_model.fit(
    train_dataset.shuffle(1000).batch(16), 
    epochs=3, 
    validation_data=val_dataset.batch(16),
    callbacks=[tensorboard_callback]
)

Epoch 1/3
 174/1250 [===>..........................] - ETA: 29:54 - loss: 0.3834 - accuracy: 0.8272

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit